In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math

import numpy as np
import pandas as pd

random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

from torch.utils.data import Dataset, DataLoader

import json
import glob
import itertools
from os.path import basename
from collections import *
from tqdm.notebook import tqdm
import networkx as nx
import random
import pickle
import tensorflow as tf
from collections import Counter
from tensorboardX import SummaryWriter

from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

import sklearn

from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split

from ast import literal_eval

In [9]:
cond_df = pd.read_csv('/shared/0/projects/condolence/working-dir/parsed//condolence_comments_2016.tsv', sep='\t')

In [10]:
neg_cond_df = pd.read_csv('/shared/0/projects/condolence/working-dir/parsed//neg_condolence_comments_2016.tsv', sep='\t')

In [12]:
cols = list(cond_df.columns)
cols[4] = 'text'
cond_df.columns = cols
cond_df['text'].head()

0    I am so incredibly sorry for your loss. There ...
1    Oh god, this is my greatest fear. I'm so sorry...
2    I am so sorry for your loss. There truly are n...
3    I'm so sorry for your loss. Can't imagine the ...
4    I am so sorry for your loss.  I don't know how...
Name: text, dtype: object

In [13]:
cols = list(neg_cond_df.columns)
cols[4] = 'text'
neg_cond_df.columns = cols
neg_cond_df['text'].head()

0    And, why has it been 18 hours and obnoxious gu...
1    15-0 would be undefeated with CCG and 4-team p...
2    Every throw certainly wasn't perfect but he ma...
3    LSU is my #1.  Ole Miss is my #2.  51 weeks ou...
4                               They probably did too.
Name: text, dtype: object

In [14]:
pos_train_df, pos_eval_df, pos_test_df = \
    np.split(cond_df.sample(frac=1), [int(.8*len(cond_df)), int(.9*len(cond_df))])

In [15]:
neg_train_df, neg_eval_df, neg_test_df = \
    np.split(neg_cond_df.sample(frac=1), [int(.8*len(neg_cond_df)), int(.9*len(neg_cond_df))])

In [16]:
print(len(neg_train_df), len(pos_train_df))

344112 344868


In [28]:
train_df = defaultdict(list)
train_df['text'].extend(pos_train_df['text'])
train_df['label'].extend([1] * len(pos_train_df['text']))
train_df['text'].extend(neg_train_df['text'])
train_df['label'].extend([0] * len(neg_train_df['text']))
train_df = pd.DataFrame(train_df)
train_df['text'] = train_df['text'].astype(str)

eval_df = defaultdict(list)
eval_df['text'].extend(pos_eval_df['text'])
eval_df['label'].extend([1] * len(pos_eval_df['text']))
eval_df['text'].extend(neg_eval_df['text'])
eval_df['label'].extend([0] * len(neg_eval_df['text']))
eval_df = pd.DataFrame(eval_df)
eval_df['text'] = eval_df['text'].astype(str)

print(len(train_df), len(eval_df))

688980 86122


In [29]:
train_df.head()

,text,label
0,Death wish coffee.,1
1,You want to know my 5 year plan? Can you tell ...,1
2,"And I completely get what you're saying, but w...",1
3,Hey there. That's also a favorite spot of mine...,1
4,Some of my friends are a little older than me....,1


In [ ]:
import warnings

def fxn():
    warnings.warn("deprecated", UserWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
    
manual_seed = 1234
train_args={
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 50,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 5000,
    "evaluate_during_training_verbose": True,
    "use_cached_eval_features": True,
    "manual_seed": manual_seed,
    'cache_dir': '/shared/0/projects/condolence/working-dir/roberta-condolence-classifier-%d' % manual_seed,
    'output_dir': '/shared/0/projects/condolence/working-dir/roberta-condolence-classifier-%d' % manual_seed
}


# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base',
                            use_cuda=True, cuda_device=3, args=train_args)
#print(train_df.head())

# Train the model
model.train_model(train_df, eval_df=eval_df, show_running_loss=True, 
                  f1=sklearn.metrics.f1_score)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

predictions, raw_outputs = model.predict([["I'd like to puts some CD-ROMS on my iPad, is that possible?'", "Yes, but wouldn't that block the screen?"]])
print(predictions)
print(raw_outputs)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/home/jurgens/.local/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:776: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/opt/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:872: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


/opt/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
